In [ ]:
import json
import sys
import os
import subprocess
import traceback
import meshio as mio
import numpy as np

executables_json = "executables.json"
normalized_path = "normalized"
embedded_path = "embedded"
offsets_path = "offsets"
c1_path = "c1"

In [ ]:
with open(executables_json, "r") as f:
    executables = json.load(f)

seamless_parametrization_binary = executables["seamless_parametrization_binary"]
polyfem_binary = executables["polyfem_binary"]
smooth_contours_binary = executables["smooth_contours_binary"]
c1_meshing_script = executables["c1_meshing_script"]

In [ ]:
def run_command(command):
    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"!!!!! Command '{command}' failed with error: {e} !!!!!")

In [ ]:
# gather files to run
input_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(embedded_path)
    for file in files
    if file.endswith(".vtu")
]
input_files

In [ ]:
# prepare directories and commands
os.makedirs(c1_path, exist_ok=True)

commands = []
for input_file in input_files:
    input_file_stem = os.path.splitext(input_file)[0]
    input_file_basename = os.path.basename(input_file)
    input_file_basename_stem = os.path.splitext(input_file_basename)[0]

    input_file = os.path.abspath(input_file)
    output_name = input_file_basename_stem

    output_folder = os.path.join(c1_path, output_name)

    if not os.path.isfile(input_file):
        print("Input file", input_file, " does not exist")

    os.makedirs(output_folder, exist_ok=True)

    log_file = os.path.join(input_file_basename_stem + "_log.txt")

    offset_file = os.path.abspath(
        os.path.join(
            offsets_path,
            output_name.split("_")[0],
            output_name.split("_")[0] + "_offsets.obj",
        )
    )
    # print(offset_file)

    # command = " ".join(
    #     [
    #         "cd",
    #         output_folder,
    #         "; ",
    #         "python3",
    #         c1_meshing_script,
    #         input_file,
    #         output_name,
    #         seamless_parametrization_binary,
    #         smooth_contours_binary,
    #         polyfem_binary,
    #         ".",
    #         offset_file,
    #         "| tee",
    #         log_file,
    #     ]
    # )

    # write spec file
    spec_file = os.path.join(output_folder, input_file_basename_stem + "_spec.json")
    spec = {"input": input_file, "output": output_name, "offset": offset_file}

    with open(spec_file, "w") as f:
        f.write(json.dumps(spec, indent=4))

    command = " ".join(
        [
            "cd",
            output_folder,
            "; ",
            "python3",
            c1_meshing_script,
            "-b",
            os.path.abspath(executables_json),
            "-j",
            os.path.abspath(spec_file),
            "| tee",
            log_file,
        ]
    )
    commands.append(command)
    break

commands

In [ ]:
for command in commands:
    run_command(command)